<a href="https://colab.research.google.com/github/udg2019/oss/blob/master/pivo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
memo = {}

def fib(n):
    if n == 0 or n == 1:
        return n

    if n not in memo:
        memo[n] = fib(n-2) + fib(n-1)

    return memo[n]

# 피보나치 수열의 5번째 항을 계산
fib(5)

# memo 내용물
print(memo)

{2: 1, 3: 2, 4: 3, 5: 5}


In [ ]:
import json
from typing import List, Dict, Tuple
from dataclasses import dataclass
import logging
from pathlib import Path

# 로깅 설정
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

@dataclass
class AnalysisResult:
    """분석 결과를 저장하는 데이터 클래스"""
    is_valid: bool
    error_type: str = ""
    error_position: int = -1
    error_details: str = ""

class FibonacciAnalyzer:
    def __init__(self):
        self.validation_errors: Dict[str, int] = {
            "wrong_start": 0,
            "wrong_sequence": 0,
            "length_error": 0,
            "non_numeric": 0
        }
        self.total_analyzed = 0
        self.correct_predictions = 0

    @staticmethod
    def is_valid_start(sequence: List[int]) -> bool:
        """피보나치 수열의 시작이 [0, 1]인지 확인"""
        return len(sequence) >= 2 and sequence[0] == 0 and sequence[1] == 1

    @staticmethod
    def verify_fibonacci_rule(sequence: List[int]) -> Tuple[bool, int]:
        """수열이 피보나치 규칙을 따르는지 검증

        Returns:
            Tuple[bool, int]: (규칙 준수 여부, 오류 발생 위치)
        """
        for i in range(2, len(sequence)):
            if sequence[i] != sequence[i-1] + sequence[i-2]:
                return False, i
        return True, -1

    def analyze_sequence(self, sequence: List[int]) -> AnalysisResult:
        """개별 수열 분석"""
        # 기본 검증
        if not sequence:
            return AnalysisResult(
                is_valid=False,
                error_type="length_error",
                error_details="Empty sequence"
            )

        if len(sequence) < 2:
            return AnalysisResult(
                is_valid=False,
                error_type="length_error",
                error_details="Sequence too short"
            )

        # 시작값 검증
        if not self.is_valid_start(sequence):
            return AnalysisResult(
                is_valid=False,
                error_type="wrong_start",
                error_details="Sequence must start with [0, 1]"
            )

        # 피보나치 규칙 검증
        is_valid, error_pos = self.verify_fibonacci_rule(sequence)
        if not is_valid:
            return AnalysisResult(
                is_valid=False,
                error_type="wrong_sequence",
                error_position=error_pos,
                error_details=f"Rule broken at position {error_pos}"
            )

        return AnalysisResult(is_valid=True)

    def analyze_test_data(self, data: List[Dict]) -> Dict:
        """테스트 데이터 전체 분석"""
        results = {
            'total_analyzed': len(data),
            'correct_predictions': 0,
            'incorrect_predictions': 0,
            'error_distribution': {},
            'detailed_errors': []
        }

        for item in data:
            sequence = item['sequence']
            claimed_fibonacci = item['is_fibonacci']
            expected_next = item['expected_next']

            # 수열 분석
            analysis = self.analyze_sequence(sequence)

            # 예측이 맞았는지 확인
            if analysis.is_valid == claimed_fibonacci:
                results['correct_predictions'] += 1
            else:
                results['incorrect_predictions'] += 1

            # 에러 통계 수집
            if not analysis.is_valid:
                error_type = analysis.error_type
                results['error_distribution'][error_type] = \
                    results['error_distribution'].get(error_type, 0) + 1

                # 상세 에러 정보 저장
                if len(results['detailed_errors']) < 10:  # 처음 10개의 에러만 저장
                    results['detailed_errors'].append({
                        'sequence': sequence,
                        'claimed_fibonacci': claimed_fibonacci,
                        'error_type': error_type,
                        'error_details': analysis.error_details
                    })

        return results

    def print_analysis_results(self, results: Dict) -> None:
        """분석 결과 출력"""
        logger.info("\n=== 분석 결과 ===")
        logger.info(f"총 분석된 데이터: {results['total_analyzed']}")
        logger.info(f"정확한 예측: {results['correct_predictions']} "
                   f"({results['correct_predictions']/results['total_analyzed']*100:.1f}%)")
        logger.info(f"부정확한 예측: {results['incorrect_predictions']} "
                   f"({results['incorrect_predictions']/results['total_analyzed']*100:.1f}%)")

        if results['error_distribution']:
            logger.info("\n에러 유형 분포:")
            for error_type, count in results['error_distribution'].items():
                logger.info(f"{error_type}: {count}개")

        if results['detailed_errors']:
            logger.info("\n상세 에러 예시 (최대 10개):")
            for i, error in enumerate(results['detailed_errors'], 1):
                logger.info(f"\n에러 케이스 {i}:")
                logger.info(f"수열: {error['sequence']}")
                logger.info(f"주장된 피보나치 여부: {error['claimed_fibonacci']}")
                logger.info(f"에러 유형: {error['error_type']}")
                logger.info(f"상세 정보: {error['error_details']}")

def main():
    try:
        # 데이터 파일 경로
        file_path = 'fibonacci_test_data.json'

        # 데이터 로드
        logger.info(f"데이터 파일 '{file_path}' 로드 중...")
        with open(file_path, 'r', encoding='utf-8') as f:
            test_data = json.load(f)

        # 분석기 생성 및 분석 수행
        analyzer = FibonacciAnalyzer()
        results = analyzer.analyze_test_data(test_data)

        # 결과 출력
        analyzer.print_analysis_results(results)

    except FileNotFoundError:
        logger.error(f"파일을 찾을 수 없습니다: {file_path}")
    except json.JSONDecodeError:
        logger.error("JSON 파일 파싱 중 오류가 발생했습니다.")
    except Exception as e:
        logger.error(f"예상치 못한 오류가 발생했습니다: {str(e)}")

if __name__ == "__main__":
    main()

In [ ]:
import random
import json
import csv
from typing import List, Dict

class FibonacciTestDataGenerator:
    def __init__(self):
        self.correct_sequences = []
        self.incorrect_sequences = []

    def generate_fibonacci_sequence(self, length: int) -> List[int]:
        """정확한 피보나치 수열을 생성합니다."""
        if length <= 0:
            return []
        elif length == 1:
            return [0]

        sequence = [0, 1]
        while len(sequence) < length:
            sequence.append(sequence[-1] + sequence[-2])
        return sequence

    def create_incorrect_sequence(self, correct_sequence: List[int]) -> List[int]:
        """올바른 피보나치 수열을 기반으로 잘못된 수열을 생성합니다."""
        incorrect = correct_sequence.copy()
        error_type = random.randint(1, 4)

        if error_type == 1:
            # 임의의 위치의 숫자를 변경
            pos = random.randint(2, len(incorrect) - 1)
            incorrect[pos] = incorrect[pos] + random.randint(1, 100)

        elif error_type == 2:
            # 순서 뒤바꾸기
            if len(incorrect) >= 4:
                pos = random.randint(2, len(incorrect) - 2)
                incorrect[pos], incorrect[pos + 1] = incorrect[pos + 1], incorrect[pos]

        elif error_type == 3:
            # 첫 두 숫자를 다른 숫자로 변경
            incorrect[0] = random.randint(1, 5)
            incorrect[1] = random.randint(1, 5)

        elif error_type == 4:
            # 마지막 숫자를 잘못된 값으로 변경
            incorrect[-1] = incorrect[-2] + incorrect[-3] + random.randint(1, 100)

        return incorrect

    def generate_test_data(self, count: int = 10000) -> List[Dict]:
        """테스트 데이터셋을 생성합니다."""
        test_data = []

        print("테스트 데이터 생성 시작...")

        for i in range(count):
            if i % 1000 == 0:
                print(f"{i}/{count} 생성 완료...")

            # 시퀀스 길이를 무작위로 선택 (3~20)
            length = random.randint(3, 20)

            # 50% 확률로 올바른/잘못된 시퀀스 생성
            is_correct = random.choice([True, False])

            if is_correct:
                sequence = self.generate_fibonacci_sequence(length)
                self.correct_sequences.append(sequence)
            else:
                correct_seq = self.generate_fibonacci_sequence(length)
                sequence = self.create_incorrect_sequence(correct_seq)
                self.incorrect_sequences.append(sequence)

            test_case = {
                'id': i + 1,
                'sequence': sequence,
                'length': len(sequence),
                'is_fibonacci': is_correct,
                'expected_next': sequence[-1] + sequence[-2] if is_correct else None
            }

            test_data.append(test_case)

        print("데이터 생성 완료!")
        return test_data

    def save_to_json(self, data: List[Dict], filename: str = 'fibonacci_test_data.json'):
        """생성된 데이터를 JSON 파일로 저장합니다."""
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(data, f, indent=2)
        print(f"데이터가 {filename}에 저장되었습니다.")

    def save_to_csv(self, data: List[Dict], filename: str = 'fibonacci_test_data.csv'):
        """생성된 데이터를 CSV 파일로 저장합니다."""
        if not data:
            return

        keys = data[0].keys()
        with open(filename, 'w', newline='', encoding='utf-8') as f:
            dict_writer = csv.DictWriter(f, keys)
            dict_writer.writeheader()
            dict_writer.writerows(data)
        print(f"데이터가 {filename}에 저장되었습니다.")

    def print_statistics(self, data: List[Dict]):
        """생성된 데이터의 통계를 출력합니다."""
        total_count = len(data)
        correct_count = sum(1 for item in data if item['is_fibonacci'])
        incorrect_count = total_count - correct_count

        print("\n테스트 데이터 통계:")
        print(f"총 테스트 케이스: {total_count}")
        print(f"올바른 피보나치 수열: {correct_count} ({correct_count/total_count*100:.1f}%)")
        print(f"잘못된 피보나치 수열: {incorrect_count} ({incorrect_count/total_count*100:.1f}%)")
        print(f"평균 수열 길이: {sum(item['length'] for item in data)/total_count:.1f}")

        print("\n올바른 수열 예시:")
        print(self.correct_sequences[0])
        print("\n잘못된 수열 예시:")
        print(self.incorrect_sequences[0])

def main():
    # 테스트 데이터 생성기 인스턴스 생성
    generator = FibonacciTestDataGenerator()

    # 10,000개의 테스트 데이터 생성
    test_data = generator.generate_test_data(10000)

    # JSON 파일로 저장
    generator.save_to_json(test_data)

    # CSV 파일로 저장
    generator.save_to_csv(test_data)

    # 통계 출력
    generator.print_statistics(test_data)

if __name__ == "__main__":
    main()

테스트 데이터 생성 시작...
0/10000 생성 완료...
1000/10000 생성 완료...
2000/10000 생성 완료...
3000/10000 생성 완료...
4000/10000 생성 완료...
5000/10000 생성 완료...
6000/10000 생성 완료...
7000/10000 생성 완료...
8000/10000 생성 완료...
9000/10000 생성 완료...
데이터 생성 완료!
데이터가 fibonacci_test_data.json에 저장되었습니다.
데이터가 fibonacci_test_data.csv에 저장되었습니다.

테스트 데이터 통계:
총 테스트 케이스: 10000
올바른 피보나치 수열: 4995 (50.0%)
잘못된 피보나치 수열: 5005 (50.0%)
평균 수열 길이: 11.6

올바른 수열 예시:
[0, 1, 1, 2, 3, 5]

잘못된 수열 예시:
[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2645]
